<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
    <i>Aprender una técnica no es un fin en si mismo, simplemente indica por donde hay que empezar</i><br><b>Proverbio Japonés</b></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<i>Material curado y organizado por el Prof. Dr. Hernán D. Merlino</i><br> 
<i>LLM Fine Tuning 1.0 - 2024 Q3</i></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>Ajuste de modelos LLM</strong></p> 

<a id = "1.0"></a>
<p style="font-size:12px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<a href="https://www.datacamp.com/tutorial/fine-tuning-large-language-models" target=\"_blank\">Base del codigo de este Notebook adaptado de: An Introductory Guide to Fine-Tuning LLMs</a>
</p>    

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Cuando Ajustar</strong><br>
Disponibilidad de recursos<br>
Rendimiento básico de modelos previamente entrenados<br>
Complejidad y especificidad de la tarea<br>
Disponibilidad de datos específicos de la tarea<br>
</p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Cargar librerias Pandas, Numpy y Warnings</strong></p> 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os

In [3]:
import pandas as pd
#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib') 
pd.options.mode.chained_assignment = None

# En produccion eliminar, se incluye por informacion
print(f'Pandas Version: {pd.__version__}') 

Pandas Version: 2.2.3


In [4]:
import numpy as np
from datasets import load_dataset
from transformers import GPT2Tokenizer
from transformers import TrainingArguments, Trainer
from transformers import GPT2ForSequenceClassification
import evaluate
import wandb

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Seleccionar un conjunto de datos que sera la base del ajuste</strong></p> 

In [5]:
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, 'basf/productos.csv')

In [7]:
df = pd.read_csv(PATH_TO_TRAIN, delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: '../basf/productos.csv'

In [7]:
df = pd.DataFrame(df)

In [8]:
df.head(10)

,nme_product,nme_target_scientific,nme_target,nme_crop,num_applic_dosage_min,num_applic_dosage_max,dsc_unit_mesure,dsc_applic_period,num_grace_period
0,7344 VT Triple PRO,Helicoverpa gelotopoeon,Isoca bolillera,Maíz,0.0,0.0,NaN,NaN,NaN
1,7344 VT Triple PRO,Spodoptera frugiperda,Oruga militar tardía,Maíz,0.0,0.0,NaN,NaN,NaN
2,7349 VT Triple PRO,Helicoverpa gelotopoeon,Isoca bolillera,Maíz,0.0,0.0,NaN,NaN,NaN
3,7349 VT Triple PRO,Spodoptera frugiperda,Oruga militar tardía,Maíz,0.0,0.0,NaN,NaN,NaN
4,7349 VT Triple PRO,Spodoptera frugiperda,Oruga militar tardía,Maíz,0.0,0.0,NaN,NaN,NaN
5,Acrobat® MZ,Plasmopara viticola,Peronospora,Vid,3.0,0.0,kg/ha,"Aplicar en forma preventiva cuando haya condiciones predisponentes para la aparición de la enfermedad, o bien desde la floración.",NaN
6,Acrobat® MZ,Phytophthora infestans,Tizón tardío de la papa,Papa,0.2,0.0,kg/100 kg de papa cortada,"Aplicar preventivamente en estadío del cultivo R3-R4 (finales de floración, inicio de llenado de granos). De observarse síntomas de la enfermedad, realizar la aplicación al inicio de floración (R1).",NaN
7,Acronis®,Cercospora sojina,Mancha ojo de rana,Soja,100.0,0.0,ml/100 kg de semilla,Tratamiento de semilla antes de la siembra.,NaN
8,Acronis®,Rhizoctonia solani,Tizón de la vaina,Algodón,100.0,0.0,ml/100 kg de semilla,En condiciones de monocultivo utilizar la dosis a 150 ml/100 Kg de semilla. Aplicar a la semilla antes de la siembra. Una aplicación en tratamiento de semilla hasta 15 días antes de la siembra.,NaN
9,Acronis®,Botrytis cinerea,Botrytis,Papa,4.0,0.0,L/há,Aspersión en banda al fondo del surco durante la plantanción asperjando los tubérculos y el fondo del surco antes de cubrirlos con tierra.,NaN


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Tokenizar el nuevo conjunto de datos</strong></p> 

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
def tokenize_function(examples):
   return tokenizer(examples['nme_product'], padding="max_length", truncation=True)

In [13]:


dataset = Dataset.from_pandas(df)

NameError: name 'Dataset' is not defined

In [12]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

NameError: name 'dataset' is not defined

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Este paso se hace para reducir el tiempo de entrenamiento, a modo de ejemplo</strong></p> 

In [19]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Se inicializa el modelo original</strong></p> 

In [ ]:
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Evaluar</strong></p> 

In [21]:
metric = evaluate.load("accuracy")

In [22]:
def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Pasar los argumentos para el entrenamiento fino</strong></p> 

In [23]:
training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )

In [24]:
trainer = Trainer(
                  model=model,
                  args=training_args,
                  train_dataset=small_train_dataset,
                  eval_dataset=small_eval_dataset,
                  compute_metrics=compute_metrics,
                  )

In [ ]:
os.environ["WANDB_API_KEY"] = '1fb92cbef7cf96e3a0bf98c4e08241e5df3e033f'
wandb.init(project="LLMFineTune",)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()